## Preprocessing

In [95]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [96]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [97]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(["EIN" , "NAME"] , axis =1 , inplace= True)


In [98]:
application_df = application_df[application_df['STATUS'] == 1]

In [73]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       1
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [99]:
application_df = application_df[application_df['ASK_AMT'] == 5000]

In [101]:
application_df['ASK_AMT'].value_counts()

5000    25394
Name: ASK_AMT, dtype: int64

In [102]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     19497
T4      1309
T19      994
T5       876
T6       847
T8       660
T7       616
T10      507
T13       54
T9        15
T12       13
T2         6
Name: APPLICATION_TYPE, dtype: int64

In [103]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9' , 'T13' , 'T12' , 'T2' , 'T25' , 'T14' , 'T29' , 'T15' , 'T17' , 'T8' , 'T7' , 'T10']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       19497
Other     1871
T4        1309
T19        994
T5         876
T6         847
Name: APPLICATION_TYPE, dtype: int64

In [104]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    12198
C2000     4581
C1200     3909
C2100     1577
C3000     1505
         ...  
C2380        1
C1732        1
C1728        1
C4120        1
C2150        1
Name: CLASSIFICATION, Length: 64, dtype: int64

In [105]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#application_df['CLASSIFICATION'].value_counts()
cls_to_not_replace = ["C1000" , "C2000" , "C1200" , "C3000" , "C2100"]
application_df[~application_df['CLASSIFICATION'].isin(cls_to_not_replace)]['CLASSIFICATION']


44       C7200
48       C1700
71       C7000
73       C4000
78       C7000
         ...  
33990    C1700
33994    C7000
34042    C7000
34097    C7000
34219    C7000
Name: CLASSIFICATION, Length: 1624, dtype: object

In [106]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cls_to_not_replace = ["C1000" , "C2000" , "C1200" , "C3000" , "C2100"]

classifications_to_replace = application_df[~application_df['CLASSIFICATION'].isin(cls_to_not_replace)]['CLASSIFICATION']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()



C1000    12198
C2000     4581
C1200     3909
Other     1624
C2100     1577
C3000     1505
Name: CLASSIFICATION, dtype: int64

In [107]:
# Convert categorical data to numeric with `pd.get_dummies`


cat_dummies = pd.get_dummies(application_df[["APPLICATION_TYPE",'AFFILIATION', 'USE_CASE', 'STATUS', 'CLASSIFICATION',
       'ORGANIZATION' , 'INCOME_AMT']])

ask_amount = application_df['ASK_AMT']

processed_df = pd.concat([cat_dummies , ask_amount] , axis=1)

processed_df.head()

,STATUS,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,...,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ASK_AMT
0,1,1,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,5000
2,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,5000
5,1,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,5000
9,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,5000
12,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,5000


In [108]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = processed_df

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [109]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [120]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units= 80, activation="tanh", input_dim=38))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=40, activation="tanh"))

#Third Hidden layer
#nn.add(tf.keras.layers.Dense(units=60, activation="tanh"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 80)                3120      
                                                                 
 dense_86 (Dense)            (None, 40)                3240      
                                                                 
 dense_87 (Dense)            (None, 1)                 41        
                                                                 
Total params: 6,401
Trainable params: 6,401
Non-trainable params: 0
_________________________________________________________________


In [121]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [124]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
596/596 [==============================] - 3s 4ms/step - loss: 0.5251 - accuracy: 0.7520
Epoch 2/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5252 - accuracy: 0.7515
Epoch 3/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5250 - accuracy: 0.7509
Epoch 4/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5253 - accuracy: 0.7516
Epoch 5/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5250 - accuracy: 0.7516
Epoch 6/100
596/596 [==============================] - 2s 3ms/step - loss: 0.5253 - accuracy: 0.7514
Epoch 7/100
596/596 [==============================] - 2s 3ms/step - loss: 0.5252 - accuracy: 0.7515
Epoch 8/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5250 - accuracy: 0.7518
Epoch 9/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5251 - accuracy: 0.7516
Epoch 10/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5250 - accura

In [125]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

199/199 - 0s - loss: 0.5355 - accuracy: 0.7511 - 264ms/epoch - 1ms/step
Loss: 0.5354782938957214, Accuracy: 0.7511419057846069


In [128]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')